##### Copyright 2018 The TensorFlow Authors.

Licensed under the Apache License, Version 2.0 (the "License");

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License"); { display-mode: "form" }
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Лучшая производительность с tf.function с AutoGraph

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/guide/function"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />Смотрите на TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/ru/guide/function.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Запустите в Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/ru/guide/function.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />Изучайте код на GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/ru/guide/function.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Скачайте ноутбук</a>
  </td>
</table>

TF 2.0 объединяет в себе простоту eager execution и мощь TF 1.0. В центре этого слияния находится `tf.function`, позволяющий преобразовывать подмножество синтаксиса Python в переносимые высокопроизводительные графы TensorFlow.

AutoGraph - крутая новая функция `tf.function`, которая позволяет писать код графа с использованием естественного синтаксиса Python. Список возможностей Python, которые можно использовать с AutoGraph, см. в [Возможности и ограничения AutoGraph] (https://github.com/tensorflow/tensorflow/blob/master/tensorflow/python/autograph/g3doc/reference/limitations. мкр). Для  дополнительной информации о `tf.function` см. RFC [TF 2.0: функции, а не сессии] (https://github.com/tensorflow/community/blob/master/rfcs/20180918-functions-not-sessions-20 .md). Для более подробной информации об AutoGraph, см. `tf.autograph`.

Этот учебник познакомит вас с базовыми функциями `tf.function` и AutoGraph.

## Setup

Импортируйте TensorFlow 2.0:

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals
import numpy as np

In [0]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

## Декоратор `tf.function`

Когда вы аннотируете функцию с помощью` tf.function`, вы все равно можете вызывать ее как любую другую функцию. Но она будет скомпилирована в граф, что означает, что вы получаете преимущества более быстрого выполнения, запуска на GPU или TPU или экспорта в SavedModel. 

In [0]:
@tf.function
def simple_nn_layer(x, y):
  return tf.nn.relu(tf.matmul(x, y))


x = tf.random.uniform((3, 3))
y = tf.random.uniform((3, 3))

simple_nn_layer(x, y)

Если мы посмотрим на результат аннотации, мы увидим, что это специально вызываемая функция, которая обрабатывает все взаимодействия со средой выполнения TensorFlow.

In [0]:
simple_nn_layer

Если ваш код использует несколько функций, вам не нужно аннотировать их все - любые функции, вызываемые из аннотированной функции, также будут работать в режиме графа.

In [0]:
def linear_layer(x):
  return 2 * x + 1


@tf.function
def deep_net(x):
  return tf.nn.relu(linear_layer(x))


deep_net(tf.constant((1, 2, 3)))

Функции могут быть быстрее, чем eager код, для графов с большим количеством маленьких операций. Но для графов с несколькими дорогими операциями (например, свертки) вы можете не увидеть большого ускорения.


In [0]:
import timeit
conv_layer = tf.keras.layers.Conv2D(100, 3)

@tf.function
def conv_fn(image):
  return conv_layer(image)

image = tf.zeros([1, 200, 200, 100])
# warm up
conv_layer(image); conv_fn(image)
print("Eager conv:", timeit.timeit(lambda: conv_layer(image), number=10))
print("Function conv:", timeit.timeit(lambda: conv_fn(image), number=10))
print("Обратите внимание, что для операций свертки нет большой разницы в производительности")


In [0]:
lstm_cell = tf.keras.layers.LSTMCell(10)

@tf.function
def lstm_fn(input, state):
  return lstm_cell(input, state)

input = tf.zeros([10, 10])
state = [tf.zeros([10, 10])] * 2
# warm up
lstm_cell(input, state); lstm_fn(input, state)
print("eager lstm:", timeit.timeit(lambda: lstm_cell(input, state), number=10))
print("function lstm:", timeit.timeit(lambda: lstm_fn(input, state), number=10))


## Используйте порядок выполнения Python

При использовании зависимого от данных порядка выполнения внутри` tf.function`, вы можете использовать операторы порядка выполнения Python, и AutoGraph преобразует их в соответствующие операции TensorFlow. Например, операторы `if` будут преобразованы в` tf.cond () ` если они зависят от `Tensor`-а.

В примере ниже `x` - `Tensor` но команда `if` работает как ожидалось:

In [0]:
@tf.function
def square_if_positive(x):
  if x > 0:
    x = x * x
  else:
    x = 0
  return x


print('square_if_positive(2) = {}'.format(square_if_positive(tf.constant(2))))
print('square_if_positive(-2) = {}'.format(square_if_positive(tf.constant(-2))))

Примечание: предыдущий пример использует простые условные выражения со скалярными значениями. <a href="#batching">Batching</a> обычно используется в коде для продакшна.

AutoGraph поддерживает обычные команды Python такие как `while`, `for`, `if`, `break`, `continue` и `return`, с поддержкой вложенности. Это значит что вы можете использовать `Tensor` выражения в условиях команд `while` и `if`, или итерировать по `Tensor`-у в цикле `for`.

In [0]:
@tf.function
def sum_even(items):
  s = 0
  for c in items:
    if c % 2 > 0:
      continue
    s += c
  return s


sum_even(tf.constant([10, 12, 15, 20]))

AutoGraph также предоставляет низкоуровневый API для опытных пользователей. Например, мы можем использовать его, чтобы взглянуть на сгенерированный код.

In [0]:
print(tf.autograph.to_code(sum_even.python_function))

Вот пример более сложного порядка выполнения:

In [0]:
@tf.function
def fizzbuzz(n):
  for i in tf.range(n):
    if i % 3 == 0:
      tf.print('Fizz')
    elif i % 5 == 0:
      tf.print('Buzz')
    else:
      tf.print(i)

fizzbuzz(tf.constant(15))

## Keras и AutoGraph

AutoGraph доступен по умолчанию в нединамических моделях Keras. Для получения дополнительной информации смотрите` tf.keras`.

In [0]:
class CustomModel(tf.keras.models.Model):

  @tf.function
  def call(self, input_data):
    if tf.reduce_mean(input_data) > 0:
      return input_data
    else:
      return input_data // 2


model = CustomModel()

model(tf.constant([-2, -4]))

## Побочные эффекты

Как и в активном режиме, вы можете использовать операции с побочными эффектами, такие как` tf.assign` или `tf.print`, используемые обычно внутри` tf.function`, и он будет вставлять необходимые управляющие зависимости, чтобы обеспечить их выполнение в нужном порядке.

In [0]:
v = tf.Variable(5)

@tf.function
def find_next_odd():
  v.assign(v + 1)
  if v % 2 == 0:
    v.assign(v + 1)


find_next_odd()
v

<a id="debugging"></a>

## Отладка

` tf.function` и AutoGraph работают, генерируя код и копируя его в графы TensorFlow. Этот механизм пока не поддерживает пошаговые отладчики, такие как `pdb`. Однако вы можете вызвать `tf.config.run_functions_eagerly(True)` временно включить eager execution внутри `tf.function` и использовать ваш любимый отладчик:

In [0]:
@tf.function
def f(x):
  if x > 0:
    # Попробуйте установить точку остановки тут!
    # Example:
    #   import pdb
    #   pdb.set_trace()
    x = x + 1
  return x

tf.config.experimental_run_functions_eagerly(True)

# Сейчас вы можете установить точки остановки и запустить код в отладчике.
f(tf.constant(1))

tf.config.experimental_run_functions_eagerly(False)

## Продвинутый пример: цикл обучения в графе

Предыдущий раздел показал, что AutoGraph можно использовать внутри слоев и моделей Keras. Модели Keras также могут быть использованы в коде AutoGraph.

Этот пример показывает, как обучить простую модель Keras в MNIST со всем процессом обучения - загрузкой пакетов, вычислением градиентов, обновлением параметров, вычислением точности валидации и повторением до сходимости - выполняемом в графе.

### Скачайте данные

In [0]:
def prepare_mnist_features_and_labels(x, y):
  x = tf.cast(x, tf.float32) / 255.0
  y = tf.cast(y, tf.int64)
  return x, y

def mnist_dataset():
  (x, y), _ = tf.keras.datasets.mnist.load_data()
  ds = tf.data.Dataset.from_tensor_slices((x, y))
  ds = ds.map(prepare_mnist_features_and_labels)
  ds = ds.take(20000).shuffle(20000).batch(100)
  return ds

train_dataset = mnist_dataset()

### Определите модель

In [0]:
model = tf.keras.Sequential((
    tf.keras.layers.Reshape(target_shape=(28 * 28,), input_shape=(28, 28)),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(10)))
model.build()
optimizer = tf.keras.optimizers.Adam()

### Определите цикл обучения

In [0]:
compute_loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

compute_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()


def train_one_step(model, optimizer, x, y):
  with tf.GradientTape() as tape:
    logits = model(x)
    loss = compute_loss(y, logits)

  grads = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(grads, model.trainable_variables))

  compute_accuracy(y, logits)
  return loss


@tf.function
def train(model, optimizer):
  train_ds = mnist_dataset()
  step = 0
  loss = 0.0
  accuracy = 0.0
  for x, y in train_ds:
    step += 1
    loss = train_one_step(model, optimizer, x, y)
    if step % 10 == 0:
      tf.print('Step', step, ': loss', loss, '; accuracy', compute_accuracy.result())
  return step, loss, accuracy

step, loss, accuracy = train(model, optimizer)
print('Final step', step, ': loss', loss, '; accuracy', compute_accuracy.result())

## Батчинг (разбивка на пакеты)

В реальных приложениях разбивка на пакеты важна для производительности. Лучший код для преобразования в AutoGraph - это код, в котором порядок выполнения определяется на уровне _batch_. Если вы принимаете решения на уровне отдельного _example_, попробуйте использовать API пакетной обработки для сохранения производительности.

Например, если у вас следующий код в Python:


In [0]:
def square_if_positive(x):
  return [i ** 2 if i > 0 else i for i in x]


square_if_positive(range(-5, 5))

Возможно, у вас возникнет соблазн написать его в TensorFlow следующим образом (и это будет работать!):


In [0]:
@tf.function
def square_if_positive_naive(x):
  result = tf.TensorArray(tf.int32, size=x.shape[0])
  for i in tf.range(x.shape[0]):
    if x[i] > 0:
      result = result.write(i, x[i] ** 2)
    else:
      result = result.write(i, x[i])
  return result.stack()


square_if_positive_naive(tf.range(-5, 5))

Но в этом случае, оказывается вы можете написать следующее:


In [0]:
def square_if_positive_vectorized(x):
  return tf.where(x > 0, x ** 2, x)


square_if_positive_vectorized(tf.range(-5, 5))

## Ре-трассировка

Ключевые моменты:

* Соблюдайте осторожность при вызове функций с нетензорными аргументами или с аргументами, которые изменяют размерности.
* Декорируйте функции уровня модуля и методы классов уровня модуля и избегайте декорирования локальных функций или методов.

`tf.function` может дать вам значительное ускорение относительно eager execution, цена этому более медленный первый запуск. Это потому что при первом запуске функция также *трассируется* в граф TensorFlow. Построение и оптимизация графа обычно намного медленнее его исполнения:

In [0]:
import timeit


@tf.function
def f(x, y):
  return tf.matmul(x, y)

print(
    "Первый вызов:",
    timeit.timeit(lambda: f(tf.ones((10, 10)), tf.ones((10, 10))), number=1))

print(
    "Второй вызов:",
    timeit.timeit(lambda: f(tf.ones((10, 10)), tf.ones((10, 10))), number=1))

Вы можете легко определить, когда произошла трассировка функции, добавив оператор `print` в начало функции. Поскольку любой код Python выполняется только во время трассировки, вы увидите результат `print`, при трассировке функции:

In [0]:
@tf.function
def f():
  print('Tracing!')
  tf.print('Executing')

print('Первый вызов:')
f()

print('Второй вызов:')
f()

`tf.function` может также *ре-трассировать* при вызове других нетензорных аргументов:

In [0]:
@tf.function
def f(n):
  print(n, 'Трассировка!')
  tf.print(n, 'Выполненние')

f(1)
f(1)

f(2)
f(2)

*Ре-трассировка* может также произойти когда тензорные аргументы поменяли размерность, если вы не указали `input_signature`:

In [0]:
@tf.function
def f(x):
  print(x.shape, 'Трассировка!')
  tf.print(x, 'Выполненние')

f(tf.constant([1]))
f(tf.constant([2]))

f(tf.constant([1, 2]))
f(tf.constant([3, 4]))

В дополнение, tf.function always всегда создает новую функцию графа со своим наборов трассировок при каждом вызове:

In [0]:
def f():
  print('Трассировка!')
  tf.print('Выполнение')

tf.function(f)()
tf.function(f)()

Это может привести к неожиданному поведению при использовании декоратора `@tf.function` во вложенной функции:

In [0]:
def outer():
  @tf.function
  def f():
    print('Трассировка!')
    tf.print('Выполнение')
  f()

outer()
outer()